import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from pathlib import Path
import warnings
warnings.filterwarnings('ignore')
## 🔍 Data Structure Analysis

Before we can extract clean election data, we need to understand how the Excel files are structured:

- **Different years** may have different layouts
- **Headers and metadata** appear before actual party data
- **Column positions** for party names and vote counts may vary
- **2024 files** are larger and more complex than historical files

This inspection will help us identify:
- Where party names are located
- Which columns contain vote totals
- How to skip header/metadata rows
- Data validation patterns

**Expected findings:**
- Party data likely starts around row 10-20
- Vote totals in rightmost columns
- Provincial breakdowns vs national totals
print("🇿🇦 SA Election Data Processing")
print("=" * 50)
print("📊 Processing South African Election Data (2009-2024)")
print("🔧 Notebook-based data processing and validation")

# Set display options
pd.set_option('display.max_columns', None)
pd.set_option('display.width', 1000)

In [3]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from pathlib import Path
import warnings
warnings.filterwarnings('ignore')

print("🇿🇦 SA Election Data Processing")
print("=" * 50)
print("📊 Processing South African Election Data (2009-2024)")
print("🔧 Notebook-based data processing and validation")

# Set display options
pd.set_option('display.max_columns', None)
pd.set_option('display.width', 1000)

🇿🇦 SA Election Data Processing
📊 Processing South African Election Data (2009-2024)
🔧 Notebook-based data processing and validation


In [5]:
# Data File Verification

# Define file structure
raw_data_dir = Path("../data/raw")
processed_data_dir = Path("../data/processed")
processed_data_dir.mkdir(exist_ok=True)

file_mapping = {
    2009: "National_2009.xls",
    2014: "National_2014.xls", 
    2019: "National_2019.xls",
    2024: "National_2024.xls"
}

# Check data availability
print("🔍 Checking data file availability...")
availability = {}

for year, filename in file_mapping.items():
    file_path = raw_data_dir / filename
    availability[year] = file_path.exists()
    
    if availability[year]:
        size_kb = file_path.stat().st_size / 1024
        print(f"✅ {year}: {filename} ({size_kb:.1f} KB)")
    else:
        print(f"❌ {year}: {filename} - NOT FOUND")

print(f"\n📊 Available elections: {sum(availability.values())}/{len(availability)}")

🔍 Checking data file availability...
✅ 2009: National_2009.xls (31.9 KB)
✅ 2014: National_2014.xls (31.9 KB)
✅ 2019: National_2019.xls (47.9 KB)
✅ 2024: National_2024.xls (47.9 KB)

📊 Available elections: 4/4


In [6]:
# Cell 3: Inspect Raw Data Structure

def inspect_excel_structure(file_path, year):
    """Inspect Excel file structure to understand data layout"""
    print(f"\n🔍 INSPECTING {year} DATA STRUCTURE")
    print("-" * 40)
    
    df = pd.read_excel(file_path, engine='xlrd')
    print(f"📊 Shape: {df.shape} (rows x columns)")
    
    # Show first 15 rows to understand structure
    print(f"\n📋 First 15 rows (first 6 columns):")
    for i in range(min(15, len(df))):
        row_data = []
        for j in range(min(6, len(df.columns))):
            val = df.iloc[i, j]
            if pd.isna(val):
                row_data.append("NaN")
            else:
                val_str = str(val)[:25]
                row_data.append(val_str)
        print(f"Row {i:2d}: {' | '.join(row_data)}")
    
    # Look for major parties
    print(f"\n🔍 Searching for major party indicators:")
    major_parties = ['AFRICAN NATIONAL CONGRESS', 'DEMOCRATIC ALLIANCE', 'ECONOMIC FREEDOM']
    
    for party in major_parties:
        for i in range(len(df)):
            for j in range(min(10, len(df.columns))):
                cell_val = str(df.iloc[i, j]) if pd.notna(df.iloc[i, j]) else ""
                if party.lower() in cell_val.lower():
                    print(f"  Found '{party}' at Row {i}, Col {j}")
                    break

# Inspect 2024 file first (most complex)
inspect_excel_structure(raw_data_dir / file_mapping[2024], 2024)


🔍 INSPECTING 2024 DATA STRUCTURE
----------------------------------------
WARNING *** file size (49088) not 512 + multiple of sector size (512)
📊 Shape: (80, 35) (rows x columns)

📋 First 15 rows (first 6 columns):
Row  0: Results Reports | NaN | NaN | NaN | NaN | NaN
Row  1: NaN | NaN | NaN | NaN | NaN | NaN
Row  2: NaN | NaN | NaN | NaN | NaN | NaN
Row  3: NaN | NaN | NaN | Detailed Results | NaN | NaN
Row  4: NaN | NaN | NaN | NaN | NaN | NaN
Row  5: NaN | NaN | NaN | NaN | NaN | NaN
Row  6: NaN | NaN | NaN | NaN | NaN | NaN
Row  7: NaN | NaN | NaN | NaN | NaN | NaN
Row  8: NaN | NaN | NaN | NaN | NaN | NaN
Row  9: NaN | NaN | NaN | NaN | NaN | NaN
Row 10: NaN | NaN | NaN | NaN | NaN | NaN
Row 11: NaN | NaN | NaN | NaN | NaN | NaN
Row 12: NaN | NaN | NaN | NaN | NaN | NaN
Row 13: NaN | NaN | NaN | NaN | NaN | NaN
Row 14: NaN | NaN | NaN | NaN | NaN | NaN

🔍 Searching for major party indicators:
  Found 'AFRICAN NATIONAL CONGRESS' at Row 31, Col 1
  Found 'DEMOCRATIC ALLIANCE' at Ro

In [7]:
# Cell 4: Extract 2024 Election Data

def extract_2024_clean_data():
    """Extract clean party data from 2024 election file"""
    print("📊 EXTRACTING 2024 ELECTION DATA")
    print("-" * 40)
    
    # Load the file
    df = pd.read_excel(raw_data_dir / file_mapping[2024], engine='xlrd')
    
    # Based on inspection, party data starts around row 31
    # Column 1 = Party Name, Column 33 = Total Votes, Column 34 = Total %
    
    parties_data = []
    
    print("🔍 Extracting parties starting from row 31...")
    
    for i in range(31, len(df)):
        row = df.iloc[i]
        
        # Get party name from column 1
        party_name = row.iloc[1] if pd.notna(row.iloc[1]) else None
        if not party_name or len(str(party_name).strip()) < 3:
            continue
            
        party_str = str(party_name).strip()
        
        # Skip metadata rows
        skip_terms = ['registered', 'total', 'valid', 'spoilt', 'votes cast', 'voter turnout']
        if any(term in party_str.lower() for term in skip_terms):
            continue
        
        # Get total votes (column 33) and percentage (column 34)
        try:
            total_votes = row.iloc[33] if len(row) > 33 and pd.notna(row.iloc[33]) else 0
            total_percent = row.iloc[34] if len(row) > 34 and pd.notna(row.iloc[34]) else 0
            
            # Clean the numbers
            if isinstance(total_votes, str):
                total_votes = float(total_votes.replace(',', ''))
            if isinstance(total_percent, str):
                total_percent = float(total_percent.replace(',', ''))
                
            total_votes = float(total_votes) if total_votes else 0
            total_percent = float(total_percent) if total_percent else 0
            
            # Only include parties with significant votes
            if total_votes > 1000:
                parties_data.append({
                    'Year': 2024,
                    'Party_Name': party_str,
                    'Total_Votes': int(total_votes),
                    'Total_Percent': total_percent
                })
                
        except Exception as e:
            print(f"⚠️  Error processing row {i}: {e}")
            continue
    
    # Create DataFrame
    df_2024 = pd.DataFrame(parties_data)
    
    print(f"✅ Extracted {len(df_2024)} parties")
    print(f"📊 Total votes: {df_2024['Total_Votes'].sum():,}")
    
    # Show top 10 parties
    print(f"\n🏆 TOP 10 PARTIES 2024:")
    top_10 = df_2024.nlargest(10, 'Total_Votes')
    for i, (_, row) in enumerate(top_10.iterrows(), 1):
        print(f"{i:2d}. {row['Party_Name'][:30]:<30} {row['Total_Votes']:>10,} ({row['Total_Percent']:>5.2f}%)")
    
    return df_2024

# Extract 2024 data
df_2024 = extract_2024_clean_data()

📊 EXTRACTING 2024 ELECTION DATA
----------------------------------------
WARNING *** file size (49088) not 512 + multiple of sector size (512)
🔍 Extracting parties starting from row 31...
✅ Extracted 39 parties
📊 Total votes: 15,667,741

🏆 TOP 10 PARTIES 2024:
 1. AFRICAN NATIONAL CONGRESS       6,459,284 ( 0.40%)
 2. DEMOCRATIC ALLIANCE             3,506,855 ( 0.22%)
 3. UMKHONTO WESIZWE                2,344,291 ( 0.15%)
 4. ECONOMIC FREEDOM FIGHTERS       1,529,914 ( 0.10%)
 5. INKATHA FREEDOM PARTY             618,208 ( 0.04%)
 6. PATRIOTIC ALLIANCE                330,416 ( 0.02%)
 7. VRYHEIDSFRONT PLUS                219,052 ( 0.01%)
 8. UNITED DEMOCRATIC MOVEMENT         78,444 ( 0.00%)
 9. RISE MZANSI                        67,970 ( 0.00%)
10. BUILD ONE SOUTH AFRICA WITH MM     65,919 ( 0.00%)


In [8]:
# Cell 5: Fix Percentages and Validate Data

def fix_percentages_2024(df):
    """Fix percentage calculations for 2024 data"""
    print("🔧 FIXING PERCENTAGE CALCULATIONS")
    print("-" * 40)
    
    # The percentages in the raw data seem to be stored incorrectly
    # Let's recalculate them properly
    
    total_valid_votes = df['Total_Votes'].sum()
    df['Total_Percent_Corrected'] = (df['Total_Votes'] / total_valid_votes) * 100
    
    print(f"📊 Total valid votes: {total_valid_votes:,}")
    print(f"🔧 Recalculated percentages based on vote totals")
    
    # Compare original vs corrected percentages for top parties
    print(f"\n📋 PERCENTAGE COMPARISON (Top 5):")
    print(f"{'Party':<30} {'Votes':<12} {'Original %':<12} {'Corrected %':<12}")
    print("-" * 70)
    
    top_5 = df.nlargest(5, 'Total_Votes')
    for _, row in top_5.iterrows():
        party_name = row['Party_Name'][:28]
        votes = f"{row['Total_Votes']:,}"
        original_pct = f"{row['Total_Percent']:.2f}%"
        corrected_pct = f"{row['Total_Percent_Corrected']:.2f}%"
        print(f"{party_name:<30} {votes:<12} {original_pct:<12} {corrected_pct:<12}")
    
    # Update the main percentage column
    df['Total_Percent'] = df['Total_Percent_Corrected']
    df = df.drop('Total_Percent_Corrected', axis=1)
    
    return df

def validate_election_data(df, year):
    """Validate the extracted election data"""
    print(f"\n✅ VALIDATING {year} ELECTION DATA")
    print("-" * 40)
    
    total_votes = df['Total_Votes'].sum()
    total_parties = len(df)
    largest_party = df.nlargest(1, 'Total_Votes').iloc[0]
    
    print(f"📊 Total votes: {total_votes:,}")
    print(f"🎉 Total parties: {total_parties}")
    print(f"🏆 Largest party: {largest_party['Party_Name']} ({largest_party['Total_Percent']:.2f}%)")
    
    # Check if any party has majority
    has_majority = largest_party['Total_Percent'] > 50
    print(f"🏛️  Majority achieved: {'Yes' if has_majority else 'No'}")
    
    # Check data quality
    parties_over_1pct = len(df[df['Total_Percent'] > 1.0])
    parties_over_5pct = len(df[df['Total_Percent'] > 5.0])
    
    print(f"📈 Parties > 1%: {parties_over_1pct}")
    print(f"📈 Parties > 5%: {parties_over_5pct}")
    
    return {
        'total_votes': total_votes,
        'total_parties': total_parties,
        'largest_party_pct': largest_party['Total_Percent'],
        'has_majority': has_majority
    }

# Fix percentages and validate
df_2024_fixed = fix_percentages_2024(df_2024.copy())
validation_2024 = validate_election_data(df_2024_fixed, 2024)

# Update our main dataframe
df_2024 = df_2024_fixed

🔧 FIXING PERCENTAGE CALCULATIONS
----------------------------------------
📊 Total valid votes: 15,667,741
🔧 Recalculated percentages based on vote totals

📋 PERCENTAGE COMPARISON (Top 5):
Party                          Votes        Original %   Corrected % 
----------------------------------------------------------------------
AFRICAN NATIONAL CONGRESS      6,459,284    0.40%        41.23%      
DEMOCRATIC ALLIANCE            3,506,855    0.22%        22.38%      
UMKHONTO WESIZWE               2,344,291    0.15%        14.96%      
ECONOMIC FREEDOM FIGHTERS      1,529,914    0.10%        9.76%       
INKATHA FREEDOM PARTY          618,208      0.04%        3.95%       

✅ VALIDATING 2024 ELECTION DATA
----------------------------------------
📊 Total votes: 15,667,741
🎉 Total parties: 39
🏆 Largest party: AFRICAN NATIONAL CONGRESS (41.23%)
🏛️  Majority achieved: No
📈 Parties > 1%: 7
📈 Parties > 5%: 4


In [9]:
# Cell 6: Process Historical Elections

def extract_historical_data(year):
    """Extract data from historical election files (2009, 2014, 2019)"""
    print(f"\n📊 EXTRACTING {year} ELECTION DATA")
    print("-" * 40)
    
    df = pd.read_excel(raw_data_dir / file_mapping[year], engine='xlrd')
    print(f"📋 File shape: {df.shape}")
    
    parties_data = []
    
    # For historical files, scan for party data (typically starts around row 10-20)
    for i in range(5, len(df)):
        row = df.iloc[i]
        
        # Look for party name in first few columns
        party_name = None
        for j in range(min(5, len(row))):
            val = row.iloc[j]
            if pd.notna(val) and isinstance(val, str) and len(val.strip()) > 3:
                # Skip header-like text
                val_lower = val.lower()
                if not any(word in val_lower for word in ['party', 'votes', 'total', '%', 'registered', 'valid']):
                    party_name = val.strip()
                    break
        
        if not party_name:
            continue
        
        # Look for vote total in last few columns
        total_votes = 0
        for j in range(len(row)-1, max(0, len(row)-8), -1):
            try:
                val = row.iloc[j]
                if pd.notna(val):
                    # Try to convert to number
                    if isinstance(val, str):
                        val = val.replace(',', '')
                    votes = float(val)
                    if votes > 5000:  # Reasonable threshold for real parties
                        total_votes = votes
                        break
            except:
                continue
        
        # Only include parties with significant votes
        if party_name and total_votes > 5000:
            # Skip summary rows
            if not any(term in party_name.lower() for term in ['total', 'spoilt', 'spoiled', 'valid', 'cast']):
                parties_data.append({
                    'Year': year,
                    'Party_Name': party_name,
                    'Total_Votes': int(total_votes)
                })
    
    # Create DataFrame and calculate percentages
    df_year = pd.DataFrame(parties_data)
    
    if len(df_year) > 0:
        total_valid_votes = df_year['Total_Votes'].sum()
        df_year['Total_Percent'] = (df_year['Total_Votes'] / total_valid_votes) * 100
        
        print(f"✅ Extracted {len(df_year)} parties")
        print(f"📊 Total votes: {total_valid_votes:,}")
        
        # Show top 5 parties
        print(f"\n🏆 TOP 5 PARTIES {year}:")
        top_5 = df_year.nlargest(5, 'Total_Votes')
        for i, (_, row) in enumerate(top_5.iterrows(), 1):
            print(f"{i}. {row['Party_Name'][:35]:<35} {row['Total_Votes']:>9,} ({row['Total_Percent']:>5.2f}%)")
    
    return df_year

# Process all historical elections
historical_data = {}
for year in [2009, 2014, 2019]:
    historical_data[year] = extract_historical_data(year)

print(f"\n✅ HISTORICAL PROCESSING COMPLETE")
print(f"📊 Processed {len(historical_data)} historical elections")


📊 EXTRACTING 2009 ELECTION DATA
----------------------------------------
WARNING *** file size (32640) not 512 + multiple of sector size (512)
📋 File shape: (44, 33)
✅ Extracted 9 parties
📊 Total votes: 2,049,779

🏆 TOP 5 PARTIES 2009:
1. DEMOCRATIC ALLIANCE/DEMOKRATIESE AL   989,132 (48.26%)
2. AFRICAN NATIONAL CONGRESS             666,223 (32.50%)
3. CONGRESS  OF THE PEOPLE               183,763 ( 8.97%)
4. INDEPENDENT DEMOCRATS                  91,001 ( 4.44%)
5. UCDP                                   43,855 ( 2.14%)

📊 EXTRACTING 2014 ELECTION DATA
----------------------------------------
WARNING *** file size (32640) not 512 + multiple of sector size (512)
📋 File shape: (53, 35)
✅ Extracted 18 parties
📊 Total votes: 17,513,340

🏆 TOP 5 PARTIES 2014:
1. AFRICAN NATIONAL CONGRESS           11,436,921 (65.30%)
2. DEMOCRATIC ALLIANCE                 4,091,584 (23.36%)
3. ECONOMIC FREEDOM FIGHTERS           1,169,259 ( 6.68%)
4. UNITED DEMOCRATIC MOVEMENT            184,636 ( 1.05%)
5

In [10]:
# Cell 7: Combine All Data and Historical Analysis

# Combine all election data
all_election_data = []

# Add historical data
for year, df in historical_data.items():
    all_election_data.append(df)

# Add 2024 data
all_election_data.append(df_2024)

# Create combined DataFrame
df_combined = pd.concat(all_election_data, ignore_index=True)

print("📊 COMBINED ELECTION DATASET")
print("=" * 50)
print(f"✅ Total records: {len(df_combined)}")
print(f"📅 Years covered: {sorted(df_combined['Year'].unique())}")
print(f"🎉 Total parties across all elections: {df_combined['Party_Name'].nunique()}")

# Create historical trends for major parties
def analyze_party_trends():
    """Analyze trends for major parties across elections"""
    print(f"\n📈 MAJOR PARTY HISTORICAL TRENDS")
    print("-" * 50)
    
    # Define major parties (handle name variations)
    major_parties = {
        'ANC': ['AFRICAN NATIONAL CONGRESS'],
        'DA': ['DEMOCRATIC ALLIANCE', 'DEMOCRATIC ALLIANCE/DEMOKRATIESE'],
        'EFF': ['ECONOMIC FREEDOM FIGHTERS'],
        'MK': ['UMKHONTO WESIZWE'],
        'IFP': ['INKATHA FREEDOM PARTY']
    }
    
    trends = {}
    
    for party_code, party_names in major_parties.items():
        trends[party_code] = {}
        
        for year in sorted(df_combined['Year'].unique()):
            year_data = df_combined[df_combined['Year'] == year]
            
            # Find party data (handle name variations)
            party_data = None
            for name_variant in party_names:
                matches = year_data[year_data['Party_Name'].str.contains(name_variant, case=False, na=False)]
                if len(matches) > 0:
                    party_data = matches.iloc[0]
                    break
            
            if party_data is not None:
                trends[party_code][year] = {
                    'votes': party_data['Total_Votes'],
                    'percent': party_data['Total_Percent']
                }
            else:
                trends[party_code][year] = {'votes': 0, 'percent': 0.0}
    
    # Display trends
    print(f"\n{'Party':<8} {'2009':<12} {'2014':<12} {'2019':<12} {'2024':<12}")
    print("-" * 60)
    
    for party_code, party_trends in trends.items():
        row_data = [party_code]
        for year in [2009, 2014, 2019, 2024]:
            if year in party_trends:
                pct = party_trends[year]['percent']
                row_data.append(f"{pct:>5.1f}%")
            else:
                row_data.append("N/A")
        
        print(f"{row_data[0]:<8} {row_data[1]:<12} {row_data[2]:<12} {row_data[3]:<12} {row_data[4]:<12}")
    
    return trends

# Calculate coalition scenarios for 2024
def analyze_2024_coalitions():
    """Analyze potential coalition scenarios for 2024"""
    print(f"\n🤝 2024 COALITION SCENARIOS")
    print("-" * 50)
    
    # Get 2024 data
    df_2024_analysis = df_combined[df_combined['Year'] == 2024].copy()
    df_2024_analysis = df_2024_analysis.sort_values('Total_Percent', ascending=False)
    
    # Major parties
    anc_pct = df_2024_analysis[df_2024_analysis['Party_Name'].str.contains('AFRICAN NATIONAL CONGRESS', case=False)]['Total_Percent'].iloc[0]
    da_pct = df_2024_analysis[df_2024_analysis['Party_Name'].str.contains('DEMOCRATIC ALLIANCE', case=False)]['Total_Percent'].iloc[0]
    mk_pct = df_2024_analysis[df_2024_analysis['Party_Name'].str.contains('UMKHONTO', case=False)]['Total_Percent'].iloc[0]
    eff_pct = df_2024_analysis[df_2024_analysis['Party_Name'].str.contains('ECONOMIC FREEDOM', case=False)]['Total_Percent'].iloc[0]
    ifp_pct = df_2024_analysis[df_2024_analysis['Party_Name'].str.contains('INKATHA', case=False)]['Total_Percent'].iloc[0]
    
    print(f"🏛️  No party achieved majority (50%+)")
    print(f"📊 ANC needs {50 - anc_pct:.1f}% additional support")
    
    # Coalition scenarios
    scenarios = [
        ("ANC + EFF", anc_pct + eff_pct),
        ("ANC + MK", anc_pct + mk_pct),
        ("ANC + IFP", anc_pct + ifp_pct),
        ("DA + MK + EFF", da_pct + mk_pct + eff_pct),
        ("ANC + DA (Unity)", anc_pct + da_pct)
    ]
    
    print(f"\n🤝 POTENTIAL COALITIONS:")
    for scenario, total_pct in scenarios:
        status = "✅ MAJORITY" if total_pct > 50 else "❌ No Majority"
        print(f"{scenario:<20} {total_pct:>5.1f}% {status}")

# Run analyses
party_trends = analyze_party_trends()
analyze_2024_coalitions()

📊 COMBINED ELECTION DATASET
✅ Total records: 95
📅 Years covered: [2009, 2014, 2019, 2024]
🎉 Total parties across all elections: 59

📈 MAJOR PARTY HISTORICAL TRENDS
--------------------------------------------------

Party    2009         2014         2019         2024        
------------------------------------------------------------
ANC       32.5%        65.3%        60.6%        41.2%      
DA        48.3%        23.4%        21.9%        22.4%      
EFF        0.0%         6.7%        11.4%         9.8%      
MK         0.0%         0.0%         0.0%        15.0%      
IFP        0.0%         0.0%         0.0%         3.9%      

🤝 2024 COALITION SCENARIOS
--------------------------------------------------
🏛️  No party achieved majority (50%+)
📊 ANC needs 8.8% additional support

🤝 POTENTIAL COALITIONS:
ANC + EFF             51.0% ✅ MAJORITY
ANC + MK              56.2% ✅ MAJORITY
ANC + IFP             45.2% ❌ No Majority
DA + MK + EFF         47.1% ❌ No Majority
ANC + DA (Unity) 

In [11]:
# Cell 8: Debug and Fix 2009 Data

def debug_2009_data():
    """Debug the 2009 data extraction to find the correct values"""
    print("🔍 DEBUGGING 2009 DATA EXTRACTION")
    print("=" * 50)
    
    df = pd.read_excel(raw_data_dir / file_mapping[2009], engine='xlrd')
    print(f"📋 2009 file shape: {df.shape}")
    
    # Search for ANC specifically
    print(f"\n🔍 Searching for ANC in 2009 data:")
    for i in range(len(df)):
        for j in range(len(df.columns)):
            cell_val = str(df.iloc[i, j]) if pd.notna(df.iloc[i, j]) else ""
            if 'african national congress' in cell_val.lower() and len(cell_val) > 10:
                print(f"Row {i}, Col {j}: {cell_val}")
                # Show surrounding data
                print(f"  Row {i} data: {[str(df.iloc[i, k]) if pd.notna(df.iloc[i, k]) else 'NaN' for k in range(min(10, len(df.columns)))]}")
    
    # Look for large vote numbers (6-7 million range for ANC)
    print(f"\n🔍 Looking for large vote totals (>5 million):")
    for i in range(len(df)):
        for j in range(len(df.columns)):
            cell_val = df.iloc[i, j]
            if pd.notna(cell_val):
                try:
                    if isinstance(cell_val, str):
                        # Remove commas and try to convert
                        num_val = float(cell_val.replace(',', ''))
                    else:
                        num_val = float(cell_val)
                    
                    if num_val > 5000000:  # Looking for millions of votes
                        print(f"Row {i}, Col {j}: {num_val:,}")
                        # Show the row context
                        row_context = []
                        for k in range(max(0, j-3), min(len(df.columns), j+4)):
                            val = df.iloc[i, k]
                            if pd.notna(val):
                                row_context.append(str(val)[:20])
                            else:
                                row_context.append("NaN")
                        print(f"  Context: {' | '.join(row_context)}")
                except:
                    continue
    
    # Show more rows around where we found data
    print(f"\n📋 Extended view of rows 10-25:")
    for i in range(10, min(25, len(df))):
        row_data = []
        for j in range(min(8, len(df.columns))):
            val = df.iloc[i, j]
            if pd.notna(val):
                val_str = str(val)[:15]
                row_data.append(val_str)
            else:
                row_data.append("NaN")
        print(f"Row {i:2d}: {' | '.join(row_data)}")

# Run the debug
debug_2009_data()

🔍 DEBUGGING 2009 DATA EXTRACTION
WARNING *** file size (32640) not 512 + multiple of sector size (512)
📋 2009 file shape: (44, 33)

🔍 Searching for ANC in 2009 data:
Row 16, Col 2: AFRICAN NATIONAL CONGRESS
  Row 16 data: ['NaN', 'NaN', 'AFRICAN NATIONAL CONGRESS', 'NaN', 'ANC', '11650748', '0.658951788696043', '1609926', '0.6970453875339175', '756287']

🔍 Looking for large vote totals (>5 million):
Row 16, Col 5: 11,650,748.0
  Context: AFRICAN NATIONAL CON | NaN | ANC | 11650748 | 0.658951788696043 | 1609926 | 0.6970453875339175
Row 40, Col 5: 17,680,729.0
  Context: Total Valid Votes | NaN | NaN | 17680729 | 1 | 2309643 | 1
Row 42, Col 5: 17,919,966.0
  Context: Total Votes Cast | NaN | NaN | 17919966 | NaN | 2344098 | NaN
Row 43, Col 5: 23,181,997.0
  Context: Registered Populatio | NaN | NaN | 23181997 | NaN | 3056559 | NaN
Row 43, Col 14: 5,555,159.0
  Context: NaN | NaN | NaN | 5555159 | NaN | NaN | NaN

📋 Extended view of rows 10-25:
Row 10: NaN | NaN | NaN | NaN | NaN | NaN | 